<a href="https://colab.research.google.com/github/Aniroodh1234/Image_Segmentation/blob/main/Image_segmentation_Watershed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 🧰 Install required libraries
!pip install opencv-python matplotlib numpy


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os


In [ ]:
# 📂 Define image folder
image_folder = "Image"

# List all images
image_files = [f for f in os.listdir(image_folder) if f.endswith(('.jpg', '.png', '.jpeg'))]

# Display original images
for img_file in image_files:
    img_path = os.path.join(image_folder, img_file)
    img = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    plt.figure(figsize=(5,5))
    plt.imshow(img_rgb)
    plt.title(f"Original Image: {img_file}")
    plt.axis('off')
    plt.show()


In [ ]:
def watershed_segmentation(image_path):
    # Step 1: Read image
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Step 2: Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Step 3: Apply threshold to get binary image
    ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Step 4: Noise removal using morphological opening
    kernel = np.ones((3,3), np.uint8)
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)

    # Step 5: Sure background area (dilate)
    sure_bg = cv2.dilate(opening, kernel, iterations=3)

    # Step 6: Sure foreground area (distance transform)
    dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
    ret, sure_fg = cv2.threshold(dist_transform, 0.7*dist_transform.max(), 255, 0)

    # Step 7: Unknown region (subtract foreground from background)
    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg, sure_fg)

    # Step 8: Marker labelling
    ret, markers = cv2.connectedComponents(sure_fg)
    markers = markers + 1
    markers[unknown==255] = 0

    # Step 9: Apply watershed
    markers_ws = cv2.watershed(img, markers)

    # Step 10: Mark boundaries in red
    img_rgb[markers_ws == -1] = [255,0,0]  # Red boundaries

    return img_rgb, markers_ws


In [ ]:
for img_file in image_files:
    img_path = os.path.join(image_folder, img_file)
    segmented_img, markers = watershed_segmentation(img_path)

    plt.figure(figsize=(10,5))
    plt.subplot(1,2,1)
    plt.imshow(cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB))
    plt.title("Original Image")
    plt.axis('off')

    plt.subplot(1,2,2)
    plt.imshow(segmented_img)
    plt.title("Watershed Segmentation")
    plt.axis('off')

    plt.suptitle(f"Watershed Segmentation - {img_file}", fontsize=14)
    plt.show()
